# N-queens problem by using hill-climbing search

In this project, you will experiment with the n-queens problem by using hill-climbing search and its variants. Read the slides or textbook carefully for the basic hill-climbing algorithm as applied to the 8-queens problem. However, your program should treat the number of queens as a variable 𝑛 and allows the user to input the value of 𝑛.

In [ ]:
import numpy as np
import time
import sys
import random
import copy

**Exception Handler**

In [ ]:
# define Python user-defined exceptions
class Error(Exception):
    """Base class for other exceptions"""
    pass


class NotInteger(Error):
    """Raised when the input value is not an integer"""
    pass


class AlreadyInputted(Error):
    """Raised when the input value is already n the list"""
    pass


class ValueOutBounds(Error):
    """Raised when the input value is not within the right range"""
    pass


## Hill climbing search

In [ ]:
class HillClimbing:
    def __init__(self, state = None, sideways_moves = 0, queens = 0):
        self.start_state = state
        

        
        
        self.queens = queens
        self.sideways_moves = sideways_moves
        self.sideways_moves_remaining = sideways_moves
        self.Number_of_steps = 0
    
    # function will return positions of queens in the given state
    def queen_positions(self,state):
        queen_pose = []
        for column, row in enumerate(state):
            queen_pose.append((row,column))
        return queen_pose
    
    # function will print an instance of the n-queen board state as a matrix
    def display_board(self, boardState):
        for r in range(self.queens):
            board_row = []
            for c in range(self.queens):
                if (r,c) in boardState:
                    board_row.append("Q")
                
                else:
                    board_row.append("*")
            print(board_row)
        print(boardState)
        print("\n\n")
    
    # function returns the contents of the cells relative to the rhs of the current cell
    def lateral_cells_adjust(self,row,column):
        j = column+1
        cells =[]
        while j < self.queens:
            cells.append((row,j))
            j = j+1
        return cells
    
    # function returns the contents of the cells relative to the diagonal rhs of the current cell
    def diag_cells_adjust(self,row,column):
        j = column+1
        cells =[]
        while j < self.queens:
            # top diagonal cells on  the right of current cell
            if row - (j-column) >= 0:
                cells.append((row-(j-column),j))
            # bottom diagonal cells on  the right of current cell
            if row + (j-column) <= self.queens - 1:
                cells.append((row+(j-column),j))
            j = j+1
        return cells
    
    # function will return all the horizontal and diagonal cells relative to the rhs of current cells
    def total_cells_to_the_right(self,row,column):
        total = self.lateral_cells_adjust(row,column) + self.diag_cells_adjust(row,column)
        return total
    
    # function returns heuristic value for a given state
    def heuristic_value(self,boardState):
        heuristic_val = 0
        for row, column in boardState:
            q_cells = set(boardState)
            r_cells = set(self.total_cells_to_the_right(row,column))            
            intersection = q_cells.intersection(r_cells)
            heuristic_val += len(intersection)
        
        return heuristic_val
    
    #  function calculates heuristic values of each cell and returns a matrix containing all the heuristic values and
    #    minimum heuristic value. Numpy lib is used here.
    def heuristic_matrix(self,boardState):
        heur_matrix = np.zeros((self.queens, self.queens), int) + (-1)
        minimum_heuristic = sum(range(self.queens))+1
        minimum_heuristic_state = None
        for (row,column) in boardState:
            for i in range(self.queens):
                if row == i:
                    pass
                else:
                    new_state = copy.deepcopy(boardState)
                    new_state[column] = temp = (i, column)
                    heur_matrix[i,column] = self.heuristic_value(new_state)                    
                    minimum_heuristic = min(minimum_heuristic, heur_matrix[i, column])
                    minimum_heuristic_state = new_state
                    
        return heur_matrix, minimum_heuristic, np.where(heur_matrix == minimum_heuristic)
    
    # function creates and returns a random state.
    def random_states(self):
        state = []
        for i in range(self.queens):
            state.append(random.randint(0, self.queens - 1))
        return state
    
    # function is a recursive implementation of the hill climbing search using steepest ascent
    def hill_climbing_search(self, state = None, heuristicVal = None, step = 0):
        boardState = None
        if(step == 0):
            state = self.start_state
            boardState = self.queen_positions(state)
            heuristicVal = self.heuristic_value(boardState)
        else:
            boardState = self.queen_positions(state)
        
        step = step + 1
        self.Number_of_steps += 1
        if heuristicVal == 0:                    
            print("Successfully finished: ")
            self.display_board(boardState)
            return 3, step

        if step == 1:
            
            print("Initial state: _")
            self.display_board(boardState)
        else:
            
            print("Step: ", step)
            self.display_board(boardState)
            
        heur_matrix = self.heuristic_matrix(boardState)
        minimum_heuristic = heur_matrix[1]
        shuffled_matrix = random.randint(0,len(heur_matrix[2][0])-1)
        shuffled_Row = heur_matrix[2][0][shuffled_matrix]
        shuffled_Col = heur_matrix[2][1][shuffled_matrix]
        newState = copy.deepcopy(state)
        newState[shuffled_Col] = shuffled_Row
        
        if minimum_heuristic < heuristicVal:
            return self.hill_climbing_search(newState, minimum_heuristic, step)
        #local Maxima condition
        elif minimum_heuristic > heuristicVal:
            return 2, step
        #flat condition
        elif minimum_heuristic == heuristicVal:
            if self.sideways_moves_remaining:
                
                self.sideways_moves_remaining -= 1
                return self.hill_climbing_search(newState, minimum_heuristic, step)
            else:
                print("The Search has failed./n")
                return 1, step
            
            
    # function implements random restart algorithm.
    def hill_climbing_random_restart(self):
        number_of_restarts = 0
        while True:
            number_of_restarts += 1
            self.start_state = self.random_states()
            print('Random-Restart start_state: ', self.start_state)
            result = self.hill_climbing_search()
            if result[0] == 3:
                return number_of_restarts, result[1], self.Number_of_steps
                break

----------------------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------------------

+ **A. Hill climbing search  ~ BoardA**

        * Run several times, say 100 to 1000,  and report success and failure rates.
        * The average number of steps when it succeeds.
        * The average number of steps when it fails.
        * The search sequences from four random initial configurations 

In [ ]:
class BoardA:
    
    def __init__(self, n, max_iterations, max_horizontal_moves = 0):
        self.n = n
        self.max_iterations = max_iterations
        self.max_horizontal_moves = max_horizontal_moves
        self.hillclimbing_stats = [[0,[]],[0,[]],[0,[]],[0,[]]]
        
        
    def analysis(self):
        
        for n in range(self.max_iterations):
            self.hillclimbing_stats[0][0] += 1
            state = []
            
            # The for loop below generates random state
            for i in range(self.n):
                state.append(random.randint(0,self.n-1))
            print("Hill climbing Search Report")
            hillClimbing = HillClimbing(state)
            result = hillClimbing.hill_climbing_search()
            self.hillclimbing_stats[result[0]][0] += 1
            self.hillclimbing_stats[result[0]][1].append(result[1])
            
        self.print_results()
        
    # prints stats of all 4 algorithms.
    def print_results(self):
        self.display_hillClimb_stats(self.hillclimbing_stats, "Hill climbing Search Report")

    # print report of the hill climbing search with and without sideways movement.
    def display_hillClimb_stats(self, result, title):
        total_number_of_Runs = result[0][0]        
        successful_runs = result[3][0]        
        if successful_runs:
            success_rate = round((successful_runs/total_number_of_Runs)*100,2)
            steps_to_success = result[3][1]
            average_steps_to_success = round(sum(steps_to_success)/successful_runs, 2)
        else:
            success_rate = steps_to_success = average_steps_to_success = '-'        
        failure_runs = result[1][0] + result[2][0]        
        if failure_runs:
            failure_rate = round((failure_runs/total_number_of_Runs)*100,2)
            steps_to_failure = result[1][1]+result[2][1]
            average_steps_to_failure = round(sum(steps_to_failure)/failure_runs,2)
        else:
            failure_rate = steps_to_failure = average_steps_to_failure = '-'        
        flat_runs = result[1][0]        
        print("\n\n")
        print(title)
        underline = ''
        for i in range(len(title)): 
            underline+="="
            
        print(underline,"\n")
        print("N(number of queens) value: ", self.n, " (i.e ",self.n,"x",self.n,")")
        print("Total number of Runs: ", total_number_of_Runs,"\n")
        print("Successful Runs: ", successful_runs)
        print("Success Rate: ", success_rate, "%")
        print("Average Steps to success: ", average_steps_to_success, "\n")
        print("Failure Runs: ", failure_runs)
        print("Failure Rate: ", failure_rate, "%")
        print("Average Steps to failure: ", average_steps_to_failure, "\n\n")
        print("Flat local maxima: ", flat_runs)        
        return

**User Input**

In [ ]:
# Getting number of queens "N" value
while(True):
    try:
        N = int(input("Enter a value for N(number of queens): "))
        if(N < 4):
            print("Enter a N(number of queens) greater than 3.")
        else:
            break
    except ValueError:
        print("Please provide a valid input")

# Getting iterations value
while(True):
    try:
        iterations = int(input("Enter a value for number of iterations: "))
        if(iterations < 1):
            print("Enter an iterations value that greater than or equal to 1.")
        else:
            break
    except ValueError:
        print("Please provide a valid input")
        
# Getting maximum sideways movement allowed value
while(True):
    try:
        lateral_movement = int(input("Enter a value for the maximum sideways moves allowed: "))
        if(lateral_movement < 1):
            print("Enter a sideways moves value that greater than or equal to 1.")
        else:
            break
    except ValueError:
        print("Please provide a valid input")
print(N)
print(iterations)
print(lateral_movement)
hc_search_report = BoardA(N, iterations, lateral_movement)
hc_search_report.analysis()

----------------------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------------------

+ **B. Hill-climbing search with sideways move  ~ BoardB**

        * Run several times, say 100 to 1000,  and report success and failure rates.
        * The average number of steps when it succeeds.
        * The average number of steps when it fails.
        * The search sequences from four random initial configurations.

In [ ]:
class BoardB:
    
    def __init__(self, n, max_iterations, max_horizontal_moves = 0):
        self.n = n
        self.max_iterations = max_iterations
        self.max_horizontal_moves = max_horizontal_moves
        self.hillclimbing_with_sideways_stats = [[0,[]],[0,[]],[0,[]],[0,[]]]
        
        
    def analysis(self):
        
        for n in range(self.max_iterations):
            self.hillclimbing_with_sideways_stats[0][0] += 1
            state = []
            
            # The for loop below generates random state
            for i in range(self.n):
                state.append(random.randint(0,self.n-1))
            
            print("Hill climbing Search with sideways movement Report")
            hillClimbing_sideways = HillClimbing(state, self.max_horizontal_moves)
            result = hillClimbing_sideways.hill_climbing_search()
            self.hillclimbing_with_sideways_stats[result[0]][0] += 1 
            self.hillclimbing_with_sideways_stats[result[0]][1].append(result[1])
            
            
        self.print_results()
        
    # print_results function displays report
    def print_results(self):
        
        self.display_hillClimb_stats(self.hillclimbing_with_sideways_stats, "Hill climbing Search with sideways movement Report")
     
    
    # print report of the hill climbing search with and without sideways movement.
    def display_hillClimb_stats(self, result, title):
        
        total_number_of_Runs = result[0][0]        
        successful_runs = result[3][0]        
        if successful_runs:
            success_rate = round((successful_runs/total_number_of_Runs)*100,2)
            steps_to_success = result[3][1]
            average_steps_to_success = round(sum(steps_to_success)/successful_runs, 2)
        else:
            success_rate = steps_to_success = average_steps_to_success = '-'        
        failure_runs = result[1][0] + result[2][0]        
        if failure_runs:
            failure_rate = round((failure_runs/total_number_of_Runs)*100,2)
            steps_to_failure = result[1][1]+result[2][1]
            average_steps_to_failure = round(sum(steps_to_failure)/failure_runs,2)
        else:
            failure_rate = steps_to_failure = average_steps_to_failure = '-'        
        flat_runs = result[1][0]        
        print("\n\n")
        print(title)
        underline = ''
        for i in range(len(title)): 
            underline+="="
            
        print(underline,"\n")
        print("N(number of queens) value: ", self.n, " (i.e ",self.n,"x",self.n,")")
        print("Total number of Runs: ", total_number_of_Runs,"\n")
        print("Successful Runs: ", successful_runs)
        print("Success Rate: ", success_rate, "%")
        print("Average Steps to success: ", average_steps_to_success, "\n")
        print("Failure Runs: ", failure_runs)
        print("Failure Rate: ", failure_rate, "%")
        print("Average Steps to failure: ", average_steps_to_failure, "\n\n")
        print("Flat local maxima: ", flat_runs)        
        return
    
    # print report of the random restart hill climbing search with and without sideways movement.
    def print_random_restart_stats(self, result, title):
        
        total_number_of_runs = result[0]
        average_number_of_restarts = sum(result[1]) / total_number_of_runs
        average_last_steps = sum(result[2]) / total_number_of_runs
        average_total_steps = sum(result[3]) / total_number_of_runs
        print("\n\n")
        print(title)
        underline = ''
        for i in range(len(title)):
            underline+="="
        print(underline, "\n")
        print("N(number of queens) value: ", self.n, " (i.e ",self.n,"x",self.n,")")
        print("Total number of Runs: ", total_number_of_runs, "\n")
        print("Average Restarts: ", average_number_of_restarts)
        print("Average Steps on last restart: ", average_last_steps)
        print("Average steps on all restarts: ", average_total_steps)

**User Input**

In [ ]:
# Getting number of queens "N" value
while(True):
    try:
        N = int(input("Enter a value for N(number of queens): "))
        if(N < 4):
            print("Enter a N(number of queens) greater than 3.")
        else:
            break
    except ValueError:
        print("Please provide a valid input")

# Getting iterations value
while(True):
    try:
        iterations = int(input("Enter a value for number of iterations: "))
        if(iterations < 1):
            print("Enter an iterations value that greater than or equal to 1.")
        else:
            break
    except ValueError:
        print("Please provide a valid input")
        
# Getting maximum sideways movement allowed value
while(True):
    try:
        lateral_movement = int(input("Enter a value for the maximum sideways moves allowed: "))
        if(lateral_movement < 1):
            print("Enter a sideways moves value that greater than or equal to 1.")
        else:
            break
    except ValueError:
        print("Please provide a valid input")

hc_search_report = BoardB(N, iterations, lateral_movement)
hc_search_report.analysis()

----------------------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------------------

+ **C. Random-restart  hill-climbing search  ~ BoardC**

        * The average number of random restarts required without sideways move.
        * The average number of steps required without sideways move.

In [ ]:
class BoardC:
    
    def __init__(self, n, max_iterations, max_horizontal_moves = 0):
        self.n = n
        self.max_iterations = max_iterations
        self.max_horizontal_moves = max_horizontal_moves
        self.random_restart_stats = [0, [], [], []]
        
        
    def analysis(self):
        if(self.n in range(4)):
            print('Invalid Number of queens(N) value. Number of queens shoule be above 3 !!!')
            return
        
        if(self.max_iterations < 1):
            print('Invalid number of iterations provided. Number of iterations should be above 1 !!!')
            return

        for n in range(self.max_iterations):
            self.random_restart_stats[0] += 1
            state = []
            
            # The for loop below generates random state
            for i in range(self.n):
                state.append(random.randint(0,self.n-1))
                        
            print("Random restart hill climbing search Analysis")
            hillClimbing_randomRestart = HillClimbing(None, 0, self.n)
            result = hillClimbing_randomRestart.hill_climbing_random_restart()
            self.random_restart_stats[1].append(result[0])
            self.random_restart_stats[2].append(result[1])
            self.random_restart_stats[3].append(result[2])
            
        self.print_results()
        
    # function prints stats of all 4 algorithms.
    def print_results(self):
        
        self.print_random_restart_stats(self.random_restart_stats, "Random restart hill climbing search Analysis")
        
    
    # print report of the hill climbing search with and without sideways movement.
    def display_hillClimb_stats(self, result, title):
        
        total_number_of_Runs = result[0][0]        
        successful_runs = result[3][0]        
        if successful_runs:
            success_rate = round((successful_runs/total_number_of_Runs)*100,2)
            steps_to_success = result[3][1]
            average_steps_to_success = round(sum(steps_to_success)/successful_runs, 2)
        else:
            success_rate = steps_to_success = average_steps_to_success = '-'        
        failure_runs = result[1][0] + result[2][0]        
        if failure_runs:
            failure_rate = round((failure_runs/total_number_of_Runs)*100,2)
            steps_to_failure = result[1][1]+result[2][1]
            average_steps_to_failure = round(sum(steps_to_failure)/failure_runs,2)
        else:
            failure_rate = steps_to_failure = average_steps_to_failure = '-'        
        flat_runs = result[1][0]        
        print("\n\n")
        print(title)
        underline = ''
        for i in range(len(title)): 
            underline+="="
            
        print(underline,"\n")
        print("N(number of queens) value: ", self.n, " (i.e ",self.n,"x",self.n,")")
        print("Total number of Runs: ", total_number_of_Runs,"\n")
        print("Successful Runs: ", successful_runs)
        print("Success Rate: ", success_rate, "%")
        print("Average Steps to success: ", average_steps_to_success, "\n")
        print("Failure Runs: ", failure_runs)
        print("Failure Rate: ", failure_rate, "%")
        print("Average Steps to failure: ", average_steps_to_failure, "\n\n")
        print("Flat local maxima: ", flat_runs)        
        return
    
    # print report of the random restart hill climbing search with and without sideways movement.
    def print_random_restart_stats(self, result, title):
        
        total_number_of_runs = result[0]
        average_number_of_restarts = sum(result[1]) / total_number_of_runs
        average_last_steps = sum(result[2]) / total_number_of_runs
        average_total_steps = sum(result[3]) / total_number_of_runs
        print("\n\n")
        print(title)
        underline = ''
        for i in range(len(title)):
            underline+="="
        print(underline, "\n")
        print("N(number of queens) value: ", self.n, " (i.e ",self.n,"x",self.n,")")
        print("Total number of Runs: ", total_number_of_runs, "\n")
        print("Average Restarts: ", average_number_of_restarts)
        print("Average Steps on last restart: ", average_last_steps)
        print("Average steps on all restarts: ", average_total_steps)

In [ ]:
# Getting number of queens "N" value
while(True):
    try:
        N = int(input("Enter a value for N(number of queens): "))
        if(N < 4):
            print("Enter a N(number of queens) greater than 3.")
        else:
            break
    except ValueError:
        print("Please provide a valid input")

# Getting iterations value
while(True):
    try:
        iterations = int(input("Enter a value for number of iterations: "))
        if(iterations < 1):
            print("Enter an iterations value that greater than or equal to 1.")
        else:
            break
    except ValueError:
        print("Please provide a valid input")
        
# Getting maximum sideways movement allowed value
while(True):
    try:
        lateral_movement = int(input("Enter a value for the maximum sideways moves allowed: "))
        if(lateral_movement < 1):
            print("Enter a sideways moves value that greater than or equal to 1.")
        else:
            break
    except ValueError:
        print("Please provide a valid input")

hc_search_report = BoardC(N, iterations, lateral_movement)
hc_search_report.analysis()

----------------------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------------------

+ **D. Random-restart  hill-climbing search  ~BoardD**

      * The average number of random restarts used with sideways move
      * The average number of steps required with sideways move

In [ ]:
class BoardD:
    
    def __init__(self, n, max_iterations, max_diag_moves = 0):
        self.n = n
        self.max_iterations = max_iterations
        self.max_diag_moves = max_diag_moves
        self.random_restart_with_sideways_stats = [0, [], [], []]
        
        
    def analysis(self):
        
        for n in range(self.max_iterations):
            self.random_restart_with_sideways_stats[0] += 1
            state = []
            
            # The for loop below generates random state
            for i in range(self.n):
                state.append(random.randint(0,self.n-1))
            print("Random restart hill climbing search with sideways movement Analysis")
            hillClimbing_randomRestart_sideways = HillClimbing(None, self.max_diag_moves, self.n)
            result = hillClimbing_randomRestart_sideways.hill_climbing_random_restart()
            self.random_restart_with_sideways_stats[1].append(result[0])
            self.random_restart_with_sideways_stats[2].append(result[1])
            self.random_restart_with_sideways_stats[3].append(result[2])
            
            
        self.print_results()
        
    #  print_results function prints stats of all 4 algorithms.
    def print_results(self):
        
        self.print_random_restart_stats(self.random_restart_with_sideways_stats, "Random restart hill climbing search with sideways movement Analysis")
    
    
    # print report of the hill climbing search with and without sideways movement.
    def display_hillClimb_stats(self, result, title):
        
        total_number_of_Runs = result[0][0]        
        successful_runs = result[3][0]        
        if successful_runs:
            success_rate = round((successful_runs/total_number_of_Runs)*100,2)
            steps_to_success = result[3][1]
            average_steps_to_success = round(sum(steps_to_success)/successful_runs, 2)
        else:
            success_rate = steps_to_success = average_steps_to_success = '-'        
        failure_runs = result[1][0] + result[2][0]        
        if failure_runs:
            failure_rate = round((failure_runs/total_number_of_Runs)*100,2)
            steps_to_failure = result[1][1]+result[2][1]
            average_steps_to_failure = round(sum(steps_to_failure)/failure_runs,2)
        else:
            failure_rate = steps_to_failure = average_steps_to_failure = '-'        
        flat_runs = result[1][0]        
        print("\n\n")
        print(title)
        underline = ''
        for i in range(len(title)): 
            underline+="="
            
        print(underline,"\n")
        print("N(number of queens) value: ", self.n, " (i.e ",self.n,"x",self.n,")")
        print("Total number of Runs: ", total_number_of_Runs,"\n")
        print("Successful Runs: ", successful_runs)
        print("Success Rate: ", success_rate, "%")
        print("Average Steps to success: ", average_steps_to_success, "\n")
        print("Failure Runs: ", failure_runs)
        print("Failure Rate: ", failure_rate, "%")
        print("Average Steps to failure: ", average_steps_to_failure, "\n\n")
        print("Flat local maxima: ", flat_runs)        
        return
    
    # print report of the random restart hill climbing search with and without sideways movement.
    def print_random_restart_stats(self, result, title):
        
        total_number_of_runs = result[0]
        average_number_of_restarts = sum(result[1]) / total_number_of_runs
        average_last_steps = sum(result[2]) / total_number_of_runs
        average_total_steps = sum(result[3]) / total_number_of_runs
        print("\n\n")
        print(title)
        underline = ''
        for i in range(len(title)):
            underline+="="
        print(underline, "\n")
        print("N(number of queens) value: ", self.n, " (i.e ",self.n,"x",self.n,")")
        print("Total number of Runs: ", total_number_of_runs, "\n")
        print("Average Restarts: ", average_number_of_restarts)
        print("Average Steps on last restart: ", average_last_steps)
        print("Average steps on all restarts: ", average_total_steps)

**User Input**

In [ ]:
N = iterations = sideways_movement = 0

# Getting number of queens "N" value
while(True):
    try:
        N = int(input("Enter a value for N(number of queens): "))
        if(N < 4):
            print("Enter a N(number of queens) greater than 3.")
        else:
            break
    except ValueError:
        print("Please provide a valid input")

# Getting iterations value
while(True):
    try:
        iterations = int(input("Enter a value for number of iterations: "))
        if(iterations < 1):
            print("Enter an iterations value that greater than or equal to 1.")
        else:
            break
    except ValueError:
        print("Please provide a valid input")
        
# Getting maximum sideways movement allowed value
while(True):
    try:
        lateral_movement = int(input("Enter a value for the maximum sideways moves allowed: "))
        if(lateral_movement < 1):
            print("Enter a sideways moves value that greater than or equal to 1.")
        else:
            break
    except ValueError:
        print("Please provide a valid input")

hc_search_report = BoardD(N, iterations, sideways_movement)
hc_search_report.analysis()